# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
from glob import glob
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [1]:
drive = 'D'

In [2]:
ods_fold = f'{drive}:/data/big2/ODS'
raw1_fold = f'{drive}:/data/big2/BD1/raw'
df1_fold = f'{drive}:/data/big2/BD1/df'
an1_fold = f'{drive}:/data/big2/BD1/an'

# Load
- 등록정보&제원정보 -> result 4등급(연료 전체)
- 정기검사&정밀검사

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [97]:
# 8.6s
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
car.info()

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898814 entries, 0 to 1898813
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   VIN                object
 1   BSPL_STDG_CD       int64 
 2   EXHST_GAS_GRD_CD   object
 3   EXHST_GAS_CERT_NO  object
 4   VHCL_ERSR_YN       object
 5   MANP_MNG_NO        object
 6   YRIDNW             int64 
 7   VHCTY_CD           object
 8   PURPS_CD2          object
 9   FRST_REG_YMD       int64 
dtypes: int64(3), object(7)
memory usage: 144.9+ MB


In [98]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
}

In [99]:
carr = car.rename(columns=car_ch_col)

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [100]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561153 entries, 0 to 561152
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   MANP_MNG_NO   561153 non-null  object
 1   FUEL_CD       518235 non-null  object
 2   VHCTY_TY_CD2  546198 non-null  object
 3   MNFCTR_NM     544020 non-null  object
 4   VHCNM         561051 non-null  object
 5   VHCL_FRM      531308 non-null  object
 6   EGIN_TY       508601 non-null  object
dtypes: object(7)
memory usage: 30.0+ MB


In [101]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [102]:
srcr = src.rename(columns=src_ch_col)

## 정기검사(STD_TB_JGT_HIS)

In [103]:
# 3m 34.9s
# jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS;")
jgt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64316863 entries, 0 to 64316862
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   VIN                     object 
 1   FDRM_INSP_INSP_MTHD_CD  object 
 2   FDRM_INSP_KND_CD        object 
 3   FDRM_INSP_JGMT          object 
 4   FDRM_NLOD_SMO_MSTVL1    float64
 5   FDRM_NLOD_SMO_MSTVL2    float64
 6   FDRM_NLOD_SMO_MSTVL3    float64
 7   FDRM_NLOD_SMO_JT_YN1    object 
 8   FDRM_INSP_YMD           int64  
 9   FDRM_DRVNG_DSTNC        float64
 10  FDRM_NLOD_SMO_PRMVL1    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 5.3+ GB


In [104]:
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}

In [105]:
jgtr = jgt.rename(columns=jgt_ch_col)
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 정밀검사(STD_TB_EET_HIS_ME)

In [106]:
# 6m 36.1s
# eet = wb.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME WHERE ROWNUM <= 10000;") # 테스트용
eet = wb.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1 FROM STD_TB_EET_HIS_ME;")
eet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99608917 entries, 0 to 99608916
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VIN                    object 
 1   PRCINSP_MSRMT_MTHD_CD  object 
 2   PRCINSP_KND_CD         object 
 3   PRCINSP_JGMT           object 
 4   PREC_NLOD_SMO_MSTVL1   float64
 5   PREC_NLOD_SMO_MSTVL2   float64
 6   PREC_NLOD_SMO_MSTVL3   float64
 7   PREC_NLOD_SMO_JT_YN1   object 
 8   PRCINSP_YMD            int64  
 9   PRCINSP_DRVNG_DSTNC    float64
 10  PREC_NLOD_SMO_PRMVL1   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.2+ GB


In [107]:
eet.columns

Index(['VIN', 'PRCINSP_MSRMT_MTHD_CD', 'PRCINSP_KND_CD', 'PRCINSP_JGMT',
       'PREC_NLOD_SMO_MSTVL1', 'PREC_NLOD_SMO_MSTVL2', 'PREC_NLOD_SMO_MSTVL3',
       'PREC_NLOD_SMO_JT_YN1', 'PRCINSP_YMD', 'PRCINSP_DRVNG_DSTNC',
       'PREC_NLOD_SMO_PRMVL1'],
      dtype='object')

In [108]:
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}

In [109]:
eetr = eet.rename(columns=eet_ch_col)
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [147]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [148]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [149]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [150]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [110]:
carr.shape, len(carr['차대번호'].unique())

((1898814, 10), 1898763)

In [111]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1898763, 10)

## 배출가스등급 코드 변환

In [112]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['4'], dtype=object)

## 차종 코드 변환

In [113]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승합', '승용', '특수', nan], dtype=object)

## 용도 코드 변환

In [114]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [115]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1515736, 10)

## 연료 코드 변환

In [116]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [117]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [118]:
# 0.7s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1515736, 16)

In [119]:
cs.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,연료,차종유형,제작사명,차명,자동차형식,엔진형식
0,KMCLK18KP8C035195,4824034000,4,NaN,N,A0810005209353308,2008,화물,개인용,20220614,경유,대형,현대자동차,메가트럭군물자수송,HD050-8MT2-LGc-A,D6GA
1,KMFLA18KP8C031274,4482533000,4,NaN,N,B1R10005100003308,2008,화물,개인용,20220419,경유,대형,신정개발특장차,신정4㎥급진공노면총소차,RS07GMS2,D6GA
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407,경유,중형,현대자동차,마이티군물자수송트럭,HD25-9MT0-LHFbF,D4GA
3,KLACA26RD8B226753,2826011300,4,NaN,N,A0710001100361207,2008,승용,개인용,20210909,경유,중형,한국지엠,윈스톰2.0S디젤,CA26R,Z20S
4,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909,경유,소형,기아자동차,봉고Ⅲ 1톤,SET42K-A,J3


## 정기&정밀 병합

In [120]:
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [121]:
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [122]:
jgtr['정기정밀'] = '정기'
eetr['정기정밀'] = '정밀'

In [123]:
# 3m 1.9s
ins = pd.concat([jgtr, eetr], ignore_index=True)
ins.shape

(163925780, 12)

### 최근 검사만 활용

In [124]:
# 4m 14.3s
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
insm.shape

(34114272, 12)

## 검사종류 코드 변환

In [125]:
# 27.5s
knd_dict = {
    'A21P01':'정밀검사', 
    'A1904':'정기검사', 
    'A21T06':'종합검사(경과)', 
    'A21T05':'종합검사', 
    'A21S01':'특정검사', 
    'A1902':'재검사(2부재검사)',
    'A21R02':'2부재검사(종합)', 
    'A21P02':'정밀검사(경과)', 
    'A1903':'정기(경과)검사', 
    'A21S04':'특정재검사(배출)', 
    'A21E01':'배출재검사', 
    'A21R01':'1부재검사(종합)',
    'A21C02':'구조변경검사', 
    'A21002':'관능재검사(임시)', 
    'A21S03':'특정재검사(관능)', 
    'A21S02':'특정검사(경과)', 
    'A1901':'재검사(1부재검사)', 
    'A21T02':'종합(정밀)',
    'A21E02':'배출재검사(구변)', 
    'A21T01':'종합(경과:정밀)', 
    'A21T04':'종합(정밀) 2부재검사', 
    'A21001':'관능재검사', 
    'A21T03':'종합(정밀) 1부재검사', 
    'A21TMP':'임시검사', 
}
insm['검사종류'] = insm['검사종류'].replace(knd_dict)
insm['검사종류'].unique()

array(['종합검사', '종합검사(경과)', '2부재검사(종합)', '1부재검사(종합)', '정기(경과)검사', '정기검사',
       '재검사(2부재검사)', '재검사(1부재검사)', nan, '배출재검사(구변)', '구조변경검사',
       '종합(정밀) 1부재검사', '종합(정밀) 2부재검사', '종합(정밀)', '종합(경과:정밀)', '정밀검사',
       '특정검사', '특정재검사(배출)', '임시검사', '배출재검사', '관능재검사', '특정재검사(관능)',
       '정밀검사(경과)', '특정검사(경과)', 'A21R03', 'A21R05', 'A21R06', '관능재검사(임시)',
       'A21E03'], dtype=object)

## 검사방법 코드 변환

In [126]:
# 14.9s
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
insm['검사방법'] = insm['검사방법'].replace(mth_dict)
insm['검사방법'].unique()

array(['무부하검사(급가속)', '부하검사(ASM-Idling)', '부하검사(KD-147)', '부하검사(LUG DOWN)',
       '무부하검사(TSI)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

## 등록&제원&정기&정밀 병합

In [141]:
# 2m 0.5s
csi = cs.merge(insm, on='차대번호', how='left')
csi.shape

(1515736, 27)

In [142]:
csi.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀'],
      dtype='object')

In [143]:
csi.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,검사종류,검사판정,무부하매연측정치1,무부하매연측정치2,무부하매연측정치3,무부하매연판정1,검사일자,주행거리,무부하매연허용치1,정기정밀
0,KMCLK18KP8C035195,4824034000,4,NaN,N,A0810005209353308,2008,화물,개인용,20220614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFLA18KP8C031274,4482533000,4,NaN,N,B1R10005100003308,2008,화물,개인용,20220419,...,종합검사,Y,7.00,1.00,0.00,Y,20230330.00,4774.00,15.00,정밀
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407,...,정기(경과)검사,Y,5.00,NaN,NaN,Y,20230523.00,79425.00,20.00,정기
3,KLACA26RD8B226753,2826011300,4,NaN,N,A0710001100361207,2008,승용,개인용,20210909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909,...,2부재검사(종합),Y,14.00,NaN,NaN,Y,20220810.00,50589.00,15.00,정밀


## 등록&제원&정기&정밀&법정동코드 병합

In [144]:
csi.shape

(1515736, 27)

In [151]:
df = csi.merge(coder, on='법정동코드', how='left')

In [152]:
df.shape

(1515736, 29)

In [159]:
df['시도'].isnull().sum()

1

In [160]:
df.loc[df['시도'].isnull(), '법정동코드']

1403523    4163055000
Name: 법정동코드, dtype: int64

In [161]:
# 주소 수정
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

# STD_BD_GRD4_CAR_CURSTT

## 4등급차만 추출

In [162]:
# 2.8s
df1 = df[df['배출가스등급'] == '4'].reset_index(drop=True)
df1.shape

(1515736, 29)

In [163]:
df1.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,무부하매연측정치1,무부하매연측정치2,무부하매연측정치3,무부하매연판정1,검사일자,주행거리,무부하매연허용치1,정기정밀,시도,시군구
0,KMCLK18KP8C035195,4824034000,4,NaN,N,A0810005209353308,2008,화물,개인용,20220614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,경상남도,사천시
1,KMFLA18KP8C031274,4482533000,4,NaN,N,B1R10005100003308,2008,화물,개인용,20220419,...,7.00,1.00,0.00,Y,20230330.00,4774.00,15.00,정밀,충청남도,태안군
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407,...,5.00,NaN,NaN,Y,20230523.00,79425.00,20.00,정기,전라북도,정읍시
3,KLACA26RD8B226753,2826011300,4,NaN,N,A0710001100361207,2008,승용,개인용,20210909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시,서구
4,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909,...,14.00,NaN,NaN,Y,20220810.00,50589.00,15.00,정밀,대구광역시,달성군


### 기준연월 정보 추가

In [164]:
today_date = datetime.today().strftime("%Y%m")
today_date

'202306'

In [165]:
df1['기준연월'] = today_date

In [166]:
df1.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구', '기준연월'],
      dtype='object')

In [167]:
STD_BD_GRD4_CAR_CURSTT = df1[[
    '기준연월', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도',
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구',
    '차명', 
    '제작사명', 
    '배출가스등급', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '무부하매연측정치1', 
    '무부하매연판정1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    ]]
STD_BD_GRD4_CAR_CURSTT.shape

(1515736, 21)

In [168]:
STD_BD_GRD4_CAR_CURSTT.head()

,기준연월,차대번호,제원관리번호,차종,용도,차량연식,차종유형,연료,법정동코드,시도,...,차명,제작사명,배출가스등급,검사방법,검사종류,검사판정,무부하매연측정치1,무부하매연판정1,무부하매연측정치2,무부하매연측정치3
0,202306,KMCLK18KP8C035195,A0810005209353308,화물,개인용,2008,대형,경유,4824034000,경상남도,...,메가트럭군물자수송,현대자동차,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202306,KMFLA18KP8C031274,B1R10005100003308,화물,개인용,2008,대형,경유,4482533000,충청남도,...,신정4㎥급진공노면총소차,신정개발특장차,4,부하검사(LUG DOWN),종합검사,Y,7.00,Y,1.00,0.00
2,202306,KMCGK17HP9C112030,A0810005008143209,화물,개인용,2009,중형,경유,4518010400,전라북도,...,마이티군물자수송트럭,현대자동차,4,무부하검사(급가속),정기(경과)검사,Y,5.00,Y,NaN,NaN
3,202306,KLACA26RD8B226753,A0710001100361207,승용,개인용,2008,중형,경유,2826011300,인천광역시,...,윈스톰2.0S디젤,한국지엠,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202306,KNCSE01429K388374,A0110003115353108,화물,개인용,2009,소형,경유,2771026522,대구광역시,...,봉고Ⅲ 1톤,기아자동차,4,부하검사(KD-147),2부재검사(종합),Y,14.00,Y,NaN,NaN


In [169]:
ch_col_dict = {
                '기준연월':'CRTR_YM',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '차명':'VHCNM', 
                '제작사명':'MNFCTR_NM', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD',
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1', 
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2', 
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3', 

                '제작일자':'FBCTN_YMD', 
                '차종분류':'VHCTY_CL_CD',
                }

In [170]:
STD_BD_GRD4_CAR_CURSTT = STD_BD_GRD4_CAR_CURSTT.rename(columns=ch_col_dict)
STD_BD_GRD4_CAR_CURSTT.columns

Index(['CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2', 'YRIDNW',
       'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'VHCNM',
       'MNFCTR_NM', 'EXHST_GAS_GRD_CD_MOD', 'INSP_MTHD', 'INSP_KND',
       'INSP_JGMT', 'NOLOD_SMO_MEVLU1', 'NOLOD_SMO_JGMT_YN1',
       'NOLOD_SMO_MEVLU2', 'NOLOD_SMO_MEVLU3'],
      dtype='object')

In [171]:
STD_BD_GRD4_CAR_CURSTT.shape

(1515736, 21)

## [출력] STD_BD_GRD4_CAR_CURSTT

In [ ]:
# expdf = STD_BD_GRD4_CAR_CURSTT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 9s
# wbt.import_from_pandas(expdf, table_nm)

In [ ]:
# # about 8.2s
# # 4등급 차량 현황
# STD_BD_GRD4_CAR_CURSTT.to_csv(os.path.join(df1_fold,  f'STD_BD_GRD4_CAR_CURSTT.csv'), index=False)

# STD_BD_GRD4_CAR_CURSTT_TOT

In [43]:
df2 = df1.copy()

## 경유차만 추출

In [44]:
df2 = df2[df2['연료'] == '경유'].reset_index(drop=True)

## 차대번호 10자리 연식

In [45]:
df2['vin10'] = df2['차대번호'].str[9]

In [46]:
vin10_dict = {'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017}

In [47]:
df2['vin10_year'] = df2['vin10'].map(vin10_dict, na_action='ignore')

## 배인번호_수정 문자 타입으로 변경

In [48]:
df2['배출가스인증번호'] = df2['배출가스인증번호'].astype('str')

## 차대번호 17자리 샘플

In [49]:
df2y = df2.loc[df2['차대번호'].str.len() == 17].reset_index(drop=True)
df2n = df2.loc[df2['차대번호'].str.len() != 17].reset_index(drop=True)
df2y.shape, df2n.shape

((1156230, 30), (10, 30))

## 차대번호 연식과 연식 동일한 샘플

In [50]:
df3y = df2y.loc[df2y['vin10_year'] == df2y['차량연식']].reset_index(drop=True)
df3n = df2y.loc[df2y['vin10_year'] != df2y['차량연식']].reset_index(drop=True)
df3y.shape, df3n.shape

((1155671, 30), (559, 30))

# 배인번호별 분석

In [51]:
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

In [52]:
# about 31.3s
# 최적화 24m 51s -> 30.0s
total_g_df = pd.DataFrame()
groupby_col1 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법', '검사판정']
groupby_col2 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법']
for one in tqdm(df3y['배출가스인증번호'].unique()):
    # 배인번호별 df
    gas_df = df3y.loc[df3y['배출가스인증번호'] == str(one)].reset_index(drop=True)

    if gas_df.shape[0] != 0:
        # 제번별, 차형식별, 엔진형식별, 검사판정별 무부하매연측정치1 통계
        g = gas_df.groupby(groupby_col1).agg({'차대번호':'count', '무부하매연측정치1':['mean', 'min', 'max']}).pipe(flat_cols).round(2).reset_index()
        g = g.rename(columns={'배출가스인증번호':'배출가스인증번호', '차대번호/count':'대수', '무부하매연측정치1/mean':'mean', '무부하매연측정치1/min':'min', '무부하매연측정치1/max':'max'})
        # 하나의 배인번호에서 제번별 엔진형식별 비율 계산
        g['합격률(%)'] = round(g['대수'] / g.groupby(groupby_col2)['대수'].transform('sum') * 100, 2)
        # 종합 - 통계
        total_g_df = pd.concat([total_g_df, g], ignore_index=True)
    else:
        print(f'오류 배인번호 : {one}')
        pass

  0%|          | 0/207 [00:00<?, ?it/s]

100%|██████████| 207/207 [00:16<00:00, 12.54it/s]


In [56]:
total_g_df.head()

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사종류,검사방법,검사판정,대수,mean,min,max,합격률(%)
0,SOFA차량,nan,9993411b020881218,CA26R-1,Z20S,정기검사,무부하검사(급가속),Y,1,11.00,11.00,11.00,100.00
1,볼보자동차코리아,nan,00920001500141307,C71,D5244T,정기검사,무부하검사(급가속),Y,1,0.00,0.00,0.00,100.00
2,에프씨에이코리아,nan,00220004500021306,WH,642980,정기검사,무부하검사(급가속),Y,1,1.00,1.00,1.00,100.00
3,에프씨에이코리아,nan,00220005800001307,JK72,50C,정기검사,무부하검사(급가속),Y,1,3.00,3.00,3.00,100.00
4,에프씨에이코리아,nan,00220005800011307,JK72,50C,정기검사,무부하검사(급가속),Y,1,0.00,0.00,0.00,100.00


# 분석

## 열화계수(SI) 지수 계산
- SI = 측정치 / 허용치

In [57]:
df3y.shape

(1155671, 30)

In [58]:
df3y.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '기준연월', 'vin10', 'vin10_year'],
      dtype='object')

In [59]:
sidf = df3y[[
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '차량연식', 
    '차명', 
    '차종유형', 
    '제작사명', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '주행거리', 
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치1'
    ]]
sidf.shape

(1155671, 17)

In [60]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [61]:
sidf['차령'] = 2023 - sidf['차량연식']
sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_11712\2774727232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['차령'] = 2023 - sidf['차량연식']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_11712\2774727232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']


In [62]:
sidf[['차령', 'SI']].isnull().sum()

차령          0
SI    1154248
dtype: int64

In [63]:
sidf[['무부하매연측정치1', '무부하매연허용치1']].isnull().sum()

무부하매연측정치1    1153502
무부하매연허용치1    1153587
dtype: int64

In [66]:
sidf.head()

,차대번호,제원관리번호,차종,차량연식,차명,차종유형,제작사명,연료,법정동코드,배출가스인증번호,검사방법,검사종류,검사판정,주행거리,무부하매연판정1,무부하매연허용치1,무부하매연측정치1,차령,SI
0,KMCLK18KP8C035195,A0810005209353308,화물,2008,메가트럭군물자수송,대형,현대자동차,경유,4824034000,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN
1,KMFLA18KP8C031274,B1R10005100003308,화물,2008,신정4㎥급진공노면총소차,대형,신정개발특장차,경유,4482533000,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN
2,KMCGK17HP9C112030,A0810005008143209,화물,2009,마이티군물자수송트럭,중형,현대자동차,경유,4518010400,7MY-HD-24-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN
3,KLACA26RD8B226753,A0710001100361207,승용,2008,윈스톰2.0S디젤,중형,한국지엠,경유,2826011300,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN
4,KNCSE01429K388374,A0110003115353108,화물,2009,봉고Ⅲ 1톤,소형,기아자동차,경유,2771026522,7MY-KM-14-65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN


### 경유만 추출

In [67]:
sidf1 = sidf[sidf['연료'] == '경유'].reset_index(drop=True)

In [68]:
sidf1 = sidf1[[
    '차대번호', 
    '제원관리번호', 
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '검사방법',
    '주행거리', 
    '차령',
    'SI', 
    ]]

In [69]:
chc_dict = {
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차종':'VHCTY_CD', 
    '연식':'YRIDNW', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종유형':'VHCTY_TY', 
    '연료':'FUEL_CD',
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '주행거리':'DRVNG_DSTNC',
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    '무부하매연허용치1':'NOLOD_SMO_PRMT_VAL1',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '차령':'VHCAG',
    }

In [70]:
STD_BD_GRD4_SI = sidf1.rename(columns=chc_dict)
STD_BD_GRD4_SI.columns

Index(['VIN', 'MANG_MNG_NO', 'VHCNM', 'MNFCTR_NM', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'DRVNG_DSTNC', 'VHCAG', 'SI'],
      dtype='object')

In [71]:
STD_BD_GRD4_SI.head()

,VIN,MANG_MNG_NO,VHCNM,MNFCTR_NM,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,DRVNG_DSTNC,VHCAG,SI
0,KMCLK18KP8C035195,A0810005209353308,메가트럭군물자수송,현대자동차,nan,NaN,NaN,15,NaN
1,KMFLA18KP8C031274,B1R10005100003308,신정4㎥급진공노면총소차,신정개발특장차,nan,NaN,NaN,15,NaN
2,KMCGK17HP9C112030,A0810005008143209,마이티군물자수송트럭,현대자동차,7MY-HD-24-28,NaN,NaN,14,NaN
3,KLACA26RD8B226753,A0710001100361207,윈스톰2.0S디젤,한국지엠,nan,NaN,NaN,15,NaN
4,KNCSE01429K388374,A0110003115353108,봉고Ⅲ 1톤,기아자동차,7MY-KM-14-65,NaN,NaN,14,NaN


In [79]:
STD_BD_GRD4_SI.shape

(1155671, 9)

### [출력] SI 지수 정보(STD_BD_GRD4_SI)

In [ ]:
# expdf = STD_BD_GRD4_SI
# table_nm = 'STD_BD_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 6s
# wbt.import_from_pandas(expdf, table_nm)

In [86]:
# # about 9.0s
# # STD_BD_GRD4_SI
# # 4등급_경유차_SI지수
# STD_BD_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_SI.csv'), index=False)

## 동일 배번에서 제번별 매연 boxplot

In [80]:
total_g_df1 = total_g_df.loc[(total_g_df['배출가스인증번호'] != '확인불가') | (total_g_df['배출가스인증번호'] != 'nan')]
total_g_df1.shape

(734, 13)

In [84]:
sample01 = total_g_df1.loc[total_g_df1['대수'] > 100].reset_index(drop=True)
sample01.shape

(734, 13)

### quantile

In [85]:
# 4m 11.2s
boxplot_df = pd.DataFrame()
quantile_df = pd.DataFrame()
for one in sample01['배출가스인증번호'].unique(): # 통계에서 100대 초과인 배인번호만 추출
    temp_one = df3y.loc[(df3y['배출가스인증번호'] == one) & (df3y['검사판정'] == 'Y')].reset_index(drop=True) # 해당 배인번호 중 검사판정이 'Y'인 샘플만 추출
    if temp_one.shape[0] > 100: # 100대 초과면 진행
        for two in temp_one['검사종류'].unique(): # 검사종류별 샘플 추출
            temp_two = temp_one.loc[temp_one['검사종류'] == two].reset_index(drop=True)
            if temp_two.shape[0] > 100:
                for three in temp_two['검사방법'].unique(): # 검사방법별 샘플 추출
                    temp_three = temp_two.loc[temp_two['검사방법'] == three].reset_index(drop=True)
                    boxplot_df = pd.concat([boxplot_df, temp_three], ignore_index=True) # 해당 샘플만 추출하여 쌓기
                    if temp_three.shape[0] > 100:
                        xticks_list= []
                        data_list = []
                        for four in temp_three['제원관리번호'].unique():
                            temp_four = temp_three.loc[temp_three['제원관리번호'] == four].reset_index(drop=True).dropna(subset=['무부하매연측정치1'])
                            if temp_four.shape[0] > 100:
                                xticks_list.append(four)
                                data_list.append(temp_four['무부하매연측정치1'])
                                temp_four['q1'] = temp_four['무부하매연측정치1'].describe()['25%']
                                temp_four['q2'] = temp_four['무부하매연측정치1'].describe()['50%']
                                temp_four['q3'] = temp_four['무부하매연측정치1'].describe()['75%']
                                quantile_df = pd.concat([quantile_df, temp_four], ignore_index=True) # 제번별 4분위 값 df형태로 저장

In [88]:
quantile_df.shape

(2096, 33)

In [89]:
quantile_df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '기준연월', 'vin10', 'vin10_year',
       'q1', 'q2', 'q3'],
      dtype='object')

In [90]:
quantile_df1 = quantile_df.drop_duplicates(['배출가스인증번호', '제원관리번호', '검사방법', '검사종류'])
quantile_df1.shape

(672, 33)

In [91]:
quantile_df2 = quantile_df1[[
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3'
    ]]
quantile_df2.shape

(672, 8)

In [92]:
chc_dict = {
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차명':'VHCNM',
                '제작사명':'MNFCTR_NM', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차종유형':'CHCTY_TY', 
                '법정동코드':'STDG_CD', 
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
                'q1':'LOWR_QRT',
                'q2':'MID_QRT',
                'q3':'UP_QRT',
                }
# '등급_수정':'EXHST_GAS_GRD_CD_MOD', 
# 'DPF유무_수정':'DPF_MNTNG_YN', 
# '시도명':'CTPV_NM', 
# '시군구명':'SGG_NM', 
# '차종분류':'VHCTY_CL_CD', 

In [93]:
STD_BD_GRD4_CAR_CURSTT_TOT = quantile_df2.rename(columns=chc_dict)
STD_BD_GRD4_CAR_CURSTT_TOT.columns

Index(['VHCNM', 'MNFCTR_NM', 'MANG_MNG_NO', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'LOWR_QRT', 'MID_QRT', 'UP_QRT'],
      dtype='object')

In [94]:
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(672, 8)

In [95]:
STD_BD_GRD4_CAR_CURSTT_TOT.head()

,VHCNM,MNFCTR_NM,MANG_MNG_NO,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,LOWR_QRT,MID_QRT,UP_QRT
0,윈스톰2.0S디젤,SOFA차량,9993411b020881218,nan,무부하검사(급가속),11.00,11.00,11.00
1,이베코6X2트랙터,한국상용차,01220001200064308,nan,무부하검사(급가속),0.00,0.00,0.00
2,짚 그랜드 체로키,에프씨에이코리아,00220004500021306,nan,무부하검사(급가속),1.00,1.00,1.00
3,짚 랭글러,에프씨에이코리아,00220005800011307,nan,무부하검사(급가속),0.00,0.00,0.00
4,이베코6X2트랙터,한국상용차,01220001000014306,nan,무부하검사(급가속),1.00,1.00,1.00


### [출력] 제번별 4분위 값 df(STD_BD_GRD4_CAR_CURSTT_TOT)

In [ ]:
# expdf = STD_BD_GRD4_CAR_CURSTT_TOT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 1s
# wbt.import_from_pandas(expdf, table_nm)

In [131]:
# # about 5.9s
# # STD_BD_GRD4_CAR_CURSTT_TOT
# # 4등급_경유_제번별_사분위수
# quantile_df2.to_csv(os.path.join(df1_fold, f'STD_BD_GRD4_CAR_CURSTT_TOT.csv'), index=False)

# code end